# Evaluation and Analysis of Pre-Trained T5 Family LLMs: Leveraging Prompt Engineering and Few-Shot Examples for Fine-Tuning

The rapid advancements in natural language processing (NLP) have underscored **the importance of leveraging pre-trained language models to enhance performance across a variety of tasks.** Among these, the T5 family of models has gained prominence due to its versatility and effectiveness in handling diverse language tasks. **This study focuses on the evaluation and analysis of pre-trained T5 models**, with a particular emphasis on utilizing prompt **engineering and few-shot examples for fine-tuning.**

Pre-trained models from the T5 family, including variants like T5-Base, T5-Large, and FLAN-T5, have demonstrated significant capabilities in text generation, question answering, and translation. However, their effectiveness can be further optimized through fine-tuning techniques that adapt these models to specific tasks or domains. Prompt engineering, a method of designing and structuring input prompts, combined with few-shot examples, offers a promising approach to enhance model performance without extensive retraining.

In this work, we systematically evaluate various T5 models, investigating how different prompt engineering strategies and few-shot learning setups impact their performance. We aim to identify best practices for fine-tuning pre-trained models to achieve better results in real-world applications. By analyzing the strengths and limitations of each model under different prompt conditions, this study provides valuable insights into optimizing T5-based LLMs for diverse NLP tasks and contributes to the broader understanding of prompt-based fine-tuning methodologies.

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate

## Setting Up Dependencies and Implementing Key Functions for T5 Model Introduction


In this section, we focus on the critical initial steps of preparing our environment and defining essential functions for working with the T5 model. This involves installing necessary libraries and implementing two key functions: `load_t5_model` and `generate_response_from_prompt`.

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

The line `from transformers import T5Tokenizer, T5ForConditionalGeneration` is used to import two essential components from the Transformers library. `T5Tokenizer` is used to **prepare text for processing by converting it into a format the T5 model can understand**. `T5ForConditionalGeneration` refers to the T5 model itself, which is designed for generating text based on the processed input. Together, these imports allow you to utilize the T5 model for various text generation tasks.

In [28]:
def load_t5_model(name):
  tokenizer_T5 = T5Tokenizer.from_pretrained(name)
  model_T5 = T5ForConditionalGeneration.from_pretrained(name, device_map="auto")
  return tokenizer_T5, model_T5

The function `load_t5_model(name)` is designed to load both the **T5 model and its tokenizer**. Here’s a breakdown of its implementation:

* `tokenizer_T5 = T5Tokenizer.from_pretrained("name"):` This line initializes the T5 tokenizer using the pre-trained `name` model. The tokenizer is responsible for converting text into a format that the **T5 model can process**.

* `model_T5 = T5ForConditionalGeneration.from_pretrained(name, device_map="auto"):` This line loads the T5 model for conditional generation, also using the `name` variant. The `device_map="auto"` argument ensures that the model is placed on the appropriate hardware device (CPU or GPU) based on availability.

* `return tokenizer_T5, model_T5:` This line returns both the tokenizer and the model as a tuple.

In summary, this function loads and returns a **pre-trained T5 tokenizer** and model, which are ready for text processing and generation tasks.

In [29]:
def generate_response_from_prompt(model, prompt, max_length=100):
  tokenizer_T5, model_T5 = load_t5_model(model)
  prompt_tokens = tokenizer_T5(prompt, return_tensors="pt").input_ids.to("cuda")
  outputs = model_T5.generate(prompt_tokens, max_length=max_length)
  return tokenizer_T5.decode(outputs[0])

The function generate_response_from_prompt is used to generate a response to a given text prompt using a **T5 model**. Here's a step-by-step breakdown of its implementation:

* `tokenizer_T5, model_FT5 = load_t5_model(model):`

This line calls the `load_t5_model` function, which returns both the **T5 tokenizer** (`tokenizer_T5`) and the **T5 model** (`model_T5`). It assumes that **load_t5_model** is designed to return these two components.

* `prompt_tokens = tokenizer_T5(prompt, return_tensors="pt").input_ids.to("cuda"):`

 - This line tokenizes the `prompt` using the **T5 tokenizer**, converting it into a format (tensor) suitable for processing by the model. The `return_tensors="pt"` argument specifies that the output should be a PyTorch tensor.

 - The tensor is then moved to the GPU `(.to("cuda"))` for faster computation, assuming that CUDA (NVIDIA's parallel computing platform) is available.

* `outputs = model_FT5.generate(prompt_tokens, max_length=max_length):`

This line generates text based on the tokenized `prompt` using the T5 model (`model_T5`). The `max_length` parameter limits the length of the generated response to the specified value.

* `return tokenizer_T5.decode(outputs[0]):`

This line decodes the generated tokens back into a human-readable text format using the **T5 tokenizer**. It returns the decoded text as the final output.

## Evaluation of a Base LLM Without Fine-Tuning: Use of T5 Base

In the realm of natural language processing, large language models have set new benchmarks in text comprehension and generation. **The T5-Base model, with 220 million parameters and available on Hugging Face**, has been pre-trained on a broad array of datasets, as detailed in its documentation. However, despite its extensive pre-training, **T5-Base has not been specifically fine-tuned for conversational tasks.**

In this section, **we will evaluate the performance of the T5-Base model in conversational contexts to examine its capabilities and limitations in dialogue scenarios.** Our goal is to demonstrate that, while T5-Base has been trained on a diverse set of data, its performance in conversational tasks can be suboptimal due to the lack of fine-tuning tailored for such applications.

Throughout this analysis, we will present test cases that illustrate how T5-Base handles conversational interactions and discuss the challenges it faces compared to models optimized for dialogue. We will evaluate aspects such as response coherence, contextual relevance, and the model's ability to maintain a natural conversation flow. Through this evaluation, we aim to highlight areas where the base T5 model may fall short in conversational use cases and suggest potential avenues for enhancing its performance through fine-tuning or additional adjustments.

In [20]:
prompt = "My name"
generate_response_from_prompt("t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> <extra_id_0> My name is Yvonne. My name is Yvonne. My name is Yvonne.</s>'

In our test, we provided the prompt **"My name"** to the model in hopes of receiving a meaningful and contextually appropriate response. However, the generated output was:

`<pad> <extra_id_0> My name is Yvonne. My name is Yvonne. My name is Yvonne.</s>`

This result reveals several key issues with the model’s performance in this instance.

* Firstly, the repetition in the `response—"My name is Yvonne"` being repeated multiple times—suggests that the model struggled to generate a coherent and varied answer. Instead of providing a concise and relevant response to the prompt, the model repeatedly generated the same phrase. This repetition can be attributed to the model’s **lack of fine-tuning for conversational tasks**, which often involves generating diverse and contextually appropriate responses.

* Secondly, the presence of special tokens like `<pad>` and `<extra_id_0>` indicates that the model’s output includes formatting elements that may not contribute directly to the meaningful content. The `<pad>` token is used for padding sequences to ensure uniform input lengths, while `<extra_id_0>` might be a placeholder for additional information. These tokens, while necessary for processing within the model, highlight that the output was not optimized for straightforward, user-facing responses.

* Finally, the model’s output ending with `</s>` meaning the end of the generated text. This tokenization detail is part of how the model structures its output but does not impact the actual content.

Overall, **the unsatisfactory result underscores that the T5-Base model**, in its base form without fine-tuning, may not be well-suited for generating high-quality, contextually relevant responses in conversational scenarios. It points to the need for more specialized fine-tuning to enhance its ability to handle prompts effectively and generate varied, coherent, and contextually accurate answers.

In [21]:
prompt = "Today is"

generate_response_from_prompt("t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> <extra_id_0> Today is the day. Today is the day.</s>'

In [23]:
prompt = "What do you think of Mars?"
generate_response_from_prompt("t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> <extra_id_0> <extra_id_1> What do you think of Mars?</s>'

In [24]:
text = """Climate change refers to significant and lasting changes in the Earth's climate \
and weather patterns over extended periods. It is primarily driven by human activities, \
especially the burning of fossil fuels such as coal, oil, and natural gas, which release \
greenhouse gases into the atmosphere. These gases, including carbon dioxide and methane, \
trap heat and lead to a warming of the planet. The consequences of climate change include \
rising sea levels, more frequent and severe extreme weather events, and disruptions to ecosystems \
and biodiversity. Addressing climate change requires global efforts to reduce greenhouse gas \
emissions, transition to renewable energy sources, and adapt to the changes that are already occurring."""

prompt = f"Summarize: {text}"

generate_response_from_prompt("t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> False</s>'

The result `"False"` from the model when summarizing the climate change text indicates that the **t5-base model** did not generate a meaningful summary. Instead, it returned an incorrect output. This suggests that the model, in its base form, **may not be well-suited for summarization tasks or may require fine-tuning to handle such prompts effectively.**

### Few-Shot Learning Technique
Few-shot learning is a method used in machine learning where a model is given a small number of examples to learn from before making predictions or generating responses. This technique is particularly useful for tasks where there isn’t a large amount of labeled data available for training.

In [36]:
review1 = """Love these plugs, have a few now. We use them to plug in lights and \
set timers to turn them on and off via a phone app. Easy to use and linked to \
the internet and apps. Good value for money."""

review2 = """Tried and tried but could never get them to work right. Too bad \
I'm past my return date or they would have gone back."""

review3 = """A well-sized, reliable smart plug. The app is easy to use and set \
up, and works well. I used them to make several lamps. Everything works fine - \
no problems."""

review4 = """Great little product. Super easy to set up. Didn't even need to use \
the Alexa app to do so. Did it with my echo. Now I use it almost daily to turn on \
a light that was a pain to get to."""

review5 = """If I could give this zero stars I would. Plug wouldn’t connect. I \
had to keep connecting it and finally just gave up and returned it. Customer service \
was a complete waste of time."""

review6 = """The smart plug works as advertised. It’s a bit bulky but it does the job \
well. The setup was straightforward, and it integrates seamlessly with my home automation system."""

prompt = f"""
Review: {review1}
Sentiment: Positive

Review: {review2}
Sentiment: Negative

Review: {review3}
Sentiment: Positive

Review: {review4}
Sentiment: Positive

Review: {review5}
Sentiment: Negative

Review: {review6}
Sentiment:
"""

generate_response_from_prompt("t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> <extra_id_0> they are great. Sentiment: Positive Review: Great plug. Easy to use and set up. Works great. Sentiment: Positive Review: Great plug. Sentiment: Positive Review: Great plug. Sentiment: Positive Review: Great plug. Sentiment: Positive Review: Great plug. Sentiment: Positive Review: Great plug. Sentiment: Positive Review: Great plug. Sentiment: Positive Review: Great plug. Sentiment:'

**In the example:**

- **Provided Reviews:** A set of reviews about smart plugs is given, each with a labeled sentiment (either Positive or Negative).

**Few-Shot Examples:**

 - Review 1: Labeled as **Positive**
 - Review 2: Labeled as **Negative**
 - Review 3: Labeled as **Positive**
 - Review 4: Labeled as **Positive**
 - Review 5: Labeled as **Negative**

These reviews serve as few-shot examples for the model. They illustrate how different sentiments are expressed in the reviews.

**Prompt Structure:**

The prompt includes these examples followed by a new review (Review 6) that the model needs to evaluate.

The task is to predict the sentiment of the new review based on the patterns observed in the few-shot examples.

**Model Execution:**

The generate_response_from_prompt function takes this structured prompt and uses the T5 model to predict the sentiment of Review 6.

The goal of providing these few-shot examples is to guide the model in understanding the sentiment classification task. By presenting a few labeled examples, the model can learn the pattern of sentiment expression and apply this understanding to the new, unlabeled review. This method leverages the model's pre-trained knowledge and fine-tunes it with minimal additional data.

## Evaluation of FLAN-T5: Advancements Over T5 and Performance Insights

The **FLAN-T5 model** represents an advanced iteration of the original T5 model, **incorporating significant improvements in performance and versatility**. While T5 has set benchmarks in natural language processing, FLAN-T5 builds upon this foundation by addressing its limitations through extensive fine-tuning. Specifically, FLAN-T5 has been refined with additional training across over 1,000 diverse tasks and extended support for multiple languages, enhancing its capability to handle a broader range of applications.

In recent evaluations, models such as Flan-PaLM 540B have demonstrated state-of-the-art performance, achieving impressive results like 75.2% on five-shot MMLU benchmarks. FLAN-T5, with its publicly available checkpoints, showcases strong few-shot learning abilities even when compared to much larger models like PaLM 62B. This performance highlights the effectiveness of instruction fine-tuning as a method to boost the performance and usability of pre-trained language models.

This section aims to evaluate FLAN-T5's performance by examining its improvements over the base T5 model, particularly focusing on its handling of few-shot learning scenarios and its overall effectiveness in real-world tasks. We will delve into various benchmarks and practical applications to assess how FLAN-T5 stands up to its predecessors and its potential for advancing NLP capabilities.

In [50]:
prompt = "My name"
generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Yvonne</s>'

In [51]:
prompt = "Today is"
generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> a day of celebration.</s>'

In [52]:
prompt = "What do you think of Mars?"
generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> good</s>'

### Text Summarization

The goal is to condense a long piece of text into a shorter summary while preserving its essential meaning. For instance, you might use a prompt to ask the model to summarize a detailed description of climate change into a concise overview.

In [53]:
text = """Climate change refers to significant and lasting changes in the Earth's climate \
and weather patterns over extended periods. It is primarily driven by human activities, \
especially the burning of fossil fuels such as coal, oil, and natural gas, which release \
greenhouse gases into the atmosphere. These gases, including carbon dioxide and methane, \
trap heat and lead to a warming of the planet. The consequences of climate change include \
rising sea levels, more frequent and severe extreme weather events, and disruptions to ecosystems \
and biodiversity. Addressing climate change requires global efforts to reduce greenhouse gas \
emissions, transition to renewable energy sources, and adapt to the changes that are already occurring."""

prompt = f"Summarize: {text}"

generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Understand climate change.</s>'

### Text Classification

This technique involves categorizing text into predefined classes based on its content. For example, you might use a prompt to classify the sentiment of a product review as positive or negative, guiding the model to understand and categorize emotional tones in the text.

In [54]:
review1 = """Love these plugs, have a few now. We use them to plug in lights and \
set timers to turn them on and off via a phone app. Easy to use and linked to \
the internet and apps. Good value for money."""

review2 = """Tried and tried but could never get them to work right. Too bad \
I'm past my return date or they would have gone back."""

review3 = """A well-sized, reliable smart plug. The app is easy to use and set \
up, and works well. I used them to make several lamps. Everything works fine - \
no problems."""

review4 = """Great little product. Super easy to set up. Didn't even need to use \
the Alexa app to do so. Did it with my echo. Now I use it almost daily to turn on \
a light that was a pain to get to."""

review5 = """If I could give this zero stars I would. Plug wouldn’t connect. I \
had to keep connecting it and finally just gave up and returned it. Customer service \
was a complete waste of time."""

review6 = """The smart plug works as advertised. It’s a bit bulky but it does the job \
well. The setup was straightforward, and it integrates seamlessly with my home automation system."""

prompt = f"""
Review: {review1}
Sentiment: Positive

Review: {review2}
Sentiment: Negative

Review: {review3}
Sentiment: Positive

Review: {review4}
Sentiment: Positive

Review: {review5}
Sentiment: Negative

Review: {review6}
Sentiment:
"""

generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Positive</s>'

### Question Answering

Here, the objective is to extract specific information from a given context in response to a query. By providing the model with a piece of text and a question about it, the model generates a precise answer based on the provided information.

In [61]:
prompt = """Given the following context, answer the question:
Context: The Amazon Rainforest is the largest tropical rainforest in the world. It spans several countries, including Brazil, Peru, and Colombia, and is home to a vast array of biodiversity. The rainforest plays a critical role in regulating the Earth's climate by absorbing carbon dioxide.
Question: What role does the Amazon Rainforest play in regulating the Earth's climate?"""

generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> absorbing carbon dioxide</s>'

### Translation

The aim is to convert text from one language to another while maintaining the original meaning. A prompt might ask the model to translate an English sentence into Spanish, demonstrating its capability to handle multilingual tasks.

In [62]:
prompt = """Translate the following English sentence to Spanish:
Sentence: "The quick brown fox jumps over the lazy dog."""

generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> El fox rojo amarillo jueve sobre el agujero lápidos.</s>'

### Creative Writing

In this case, the model generates imaginative and engaging text based on a given prompt. For example, you could ask the model to create a story starting with a specific scenario, allowing it to produce creative content that builds on the initial idea.

In [63]:
prompt = """Generate a creative story based on the following prompt:
Prompt: "Once upon a time, in a land where magic was real, a young girl discovered a hidden door in her grandmother's attic."""

generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


"<pad> A young girl discovered a hidden door in her grandmother's attic. She was a little girl. She was a little girl. She was a little girl. One day, she discovered a hidden door in her grandmother's attic. She was a little girl.</s>"

### Factual Information

The objective here is to provide a clear and concise explanation of a factual concept. By prompting the model with a straightforward query about a scientific concept like black holes, it generates a brief and accurate explanation of the topic.

In [64]:
prompt = """Provide a brief explanation of what a black hole is:
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity after their nuclear fuel is exhausted."""

generate_response_from_prompt("google/flan-t5-base", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Black holes are formed when massive stars collapse under their own gravity after their nuclear fuel is exhausted.</s>'

## Evaluation of the FLAN-T5-Large Model: Performance Analysis and Capabilities

The **FLAN-T5-Large** model represents an advanced iteration of the T5 architecture, **designed to excel beyond its predecessors in both performance and versatility.** Leveraging a larger model size and additional fine-tuning on a diverse set of tasks and languages, **FLAN-T5-Large** is optimized to tackle more complex and specialized NLP challenges effectively.

This section aims to evaluate the performance of the **FLAN-T5-Large** model across various natural language processing scenarios. We will assess how this model performs in tasks such as text generation, question answering, and machine translation, comparing its capabilities to those of previous models and its base version. With extensive fine-tuning on multiple tasks, **FLAN-T5-Large** promises enhanced performance in key benchmarks, making it a powerful tool for real-world applications.

Throughout this evaluation, detailed results will be presented on how **FLAN-T5-Large** handles different NLP tasks, highlighting its strengths and any limitations. Comparative analysis with other models will provide insight into its effectiveness in practical applications and its impact on the field of machine learning.

In [65]:
prompt = "My name"
generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> sarah</s>'

In [66]:
prompt = "Today is"
generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Today, we celebrate the birth of the first woman to hold office in the United States.</s>'

In [67]:
prompt = "What do you think of Mars?"
generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> it is a planet</s>'

### Text Summarization

The goal is to condense a long piece of text into a shorter summary while preserving its essential meaning. For instance, you might use a prompt to ask the model to summarize a detailed description of climate change into a concise overview.

In [68]:
text = """Climate change refers to significant and lasting changes in the Earth's climate \
and weather patterns over extended periods. It is primarily driven by human activities, \
especially the burning of fossil fuels such as coal, oil, and natural gas, which release \
greenhouse gases into the atmosphere. These gases, including carbon dioxide and methane, \
trap heat and lead to a warming of the planet. The consequences of climate change include \
rising sea levels, more frequent and severe extreme weather events, and disruptions to ecosystems \
and biodiversity. Addressing climate change requires global efforts to reduce greenhouse gas \
emissions, transition to renewable energy sources, and adapt to the changes that are already occurring."""

prompt = f"Summarize: {text}"

generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Understand climate change.</s>'

### Text Classification

This technique involves categorizing text into predefined classes based on its content. For example, you might use a prompt to classify the sentiment of a product review as positive or negative, guiding the model to understand and categorize emotional tones in the text.

In [69]:
review1 = """Love these plugs, have a few now. We use them to plug in lights and \
set timers to turn them on and off via a phone app. Easy to use and linked to \
the internet and apps. Good value for money."""

review2 = """Tried and tried but could never get them to work right. Too bad \
I'm past my return date or they would have gone back."""

review3 = """A well-sized, reliable smart plug. The app is easy to use and set \
up, and works well. I used them to make several lamps. Everything works fine - \
no problems."""

review4 = """Great little product. Super easy to set up. Didn't even need to use \
the Alexa app to do so. Did it with my echo. Now I use it almost daily to turn on \
a light that was a pain to get to."""

review5 = """If I could give this zero stars I would. Plug wouldn’t connect. I \
had to keep connecting it and finally just gave up and returned it. Customer service \
was a complete waste of time."""

review6 = """The smart plug works as advertised. It’s a bit bulky but it does the job \
well. The setup was straightforward, and it integrates seamlessly with my home automation system."""

prompt = f"""
Review: {review1}
Sentiment: Positive

Review: {review2}
Sentiment: Negative

Review: {review3}
Sentiment: Positive

Review: {review4}
Sentiment: Positive

Review: {review5}
Sentiment: Negative

Review: {review6}
Sentiment:
"""

generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Positive</s>'

### Question Answering

Here, the objective is to extract specific information from a given context in response to a query. By providing the model with a piece of text and a question about it, the model generates a precise answer based on the provided information.

In [70]:
prompt = """Given the following context, answer the question:
Context: The Amazon Rainforest is the largest tropical rainforest in the world. It spans several countries, including Brazil, Peru, and Colombia, and is home to a vast array of biodiversity. The rainforest plays a critical role in regulating the Earth's climate by absorbing carbon dioxide.
Question: What role does the Amazon Rainforest play in regulating the Earth's climate?"""

generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> absorbing carbon dioxide</s>'

### Translation

The aim is to convert text from one language to another while maintaining the original meaning. A prompt might ask the model to translate an English sentence into Spanish, demonstrating its capability to handle multilingual tasks.

In [71]:
prompt = """Translate the following English sentence to Spanish:
Sentence: "The quick brown fox jumps over the lazy dog."""

generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> Sentence: "El fox rojo rápido juega sobre el borracho.</s>'

### Creative Writing

In this case, the model generates imaginative and engaging text based on a given prompt. For example, you could ask the model to create a story starting with a specific scenario, allowing it to produce creative content that builds on the initial idea.

In [72]:
prompt = """Generate a creative story based on the following prompt:
Prompt: "Once upon a time, in a land where magic was real, a young girl discovered a hidden door in her grandmother's attic."""

generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


"<pad>The girl was a little girl who lived in a small town in the middle of nowhere. Her grandmother had a secret door in her attic that she had never seen before. She was so excited to find it, she hid it in her grandmother's attic. The next day, she found it in her grandmother's attic. She was so excited to find it, she hid it in her grandmother's attic. The next day, she found"

### Factual Information

The objective here is to provide a clear and concise explanation of a factual concept. By prompting the model with a straightforward query about a scientific concept like black holes, it generates a brief and accurate explanation of the topic.

In [73]:
prompt = """Provide a brief explanation of what a black hole is:
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity after their nuclear fuel is exhausted."""

generate_response_from_prompt("google/flan-t5-large", prompt)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<pad> A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it.</s>'